# Elementary Statistical Analysis and Visualization of Video Game Data

#### In this notebook I aim to grasp a better understanding of visual analysis in python by cutting my teeth on video game data scraped from the web. First I will scrape data from an online video games database. I will then munge and parse the data for use, and finally visualize several aspects of the data using python code in order to extract a variety of conclusions. While the end conclusions I reach from the data are not the overall goal of this project, I hope to come away with at least a few worthwhile ideas about games I may want to put on my "to-play" list.

## Section 0: Resources and Acknowledgements
#### I will be consulting a number of websites and textbooks for this project. I will list them below as I use them, both as credit where it's due and to give anyone who may read this a good list of resources should they embark on a similar learning endeavor:

1: As always when I need a git refresher: Pro Git by Scott Chacon and Ben Straub - available at https://git-scm.com/book/en/v2

2: Twitch GiantBomb API Documentation - available at https://www.giantbomb.com/api/



## Section 1: Querying the Data

### I will be using the GiantBomb.com games database to query data for these analyses. The documentation for the GiantBomb API can be found in Secion 0.

In [3]:
#typically I would list all of my imports in one section, but as this is largely a learning exercise I will import packages in the sections at which they become relevant.
import numpy as np 
import pandas as pd 
import requests
import json


#read credentials auth.json from project directory - this is user info and is therefore kept private - auth.json is in .gitignore
user_info = json.loads(open('./auth.json').read())

headers = {'user-agent':user_info['user-agent']} #giant bomb api rejects requests without a unique user-agent

#define query parameters
payload = {
    'api_key':user_info['api_key'],
    'format':'json',
    'field_list':'name,original_release_date',
    'sort':'',
    'filter':'',
    'limit':'5'

}


r = requests.get('https://www.giantbomb.com/api/games/', headers=headers, params=payload)


data = json.loads(r.text)
games_df = pd.DataFrame(data)

games_df






,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version
0,OK,5,0,5,76011,1,"{'name': 'Desert Strike: Return to the Gulf', ...",1.0
1,OK,5,0,5,76011,1,"{'name': 'Breakfree', 'original_release_date':...",1.0
2,OK,5,0,5,76011,1,"{'name': 'Hyperballoid Deluxe: Survival Pack',...",1.0
3,OK,5,0,5,76011,1,"{'name': 'The Chessmaster 2000', 'original_rel...",1.0
4,OK,5,0,5,76011,1,"{'name': 'Bass Avenger', 'original_release_dat...",1.0


In [4]:
print(games_df['results'][0])

{'name': 'Desert Strike: Return to the Gulf', 'original_release_date': '1992-01-01'}
